# 如何处理多个检索器

:::info 预备知识

本指南假定您熟悉以下内容：

- [查询分析](/docs/tutorials/rag#query-analysis)

:::

有时，查询分析技术可能会允许选择使用哪个检索器。要使用此功能，您需要添加一些逻辑来选择要使用的检索器。我们将展示一个简单的示例（使用模拟数据），说明如何实现这一点。

## 安裝

### 安裝依賴項

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/openai @langchain/core zod chromadb
</Npm2Yarn>
```

### 設定環境變數

```
OPENAI_API_KEY=your-api-key

# 選用，使用 LangSmith 以獲得最佳的可觀測性
LANGSMITH_API_KEY=your-api-key
LANGSMITH_TRACING=true

# 如果你不在無伺服器環境中，可減少追蹤延遲
# LANGCHAIN_CALLBACKS_BACKGROUND=true
```

### 创建索引

我们将在虚假信息上创建一个向量存储。

In [ ]:
import { Chroma } from "@langchain/community/vectorstores/chroma"
import { OpenAIEmbeddings } from "@langchain/openai"
import "chromadb";

const texts = ["Harrison worked at Kensho"]
const embeddings = new OpenAIEmbeddings({ model: "text-embedding-3-small" })
const vectorstore = await Chroma.fromTexts(texts, {}, embeddings, {
  collectionName: "harrison"
})
const retrieverHarrison = vectorstore.asRetriever(1)

const textsAnkush = ["Ankush worked at Facebook"]
const embeddingsAnkush = new OpenAIEmbeddings({ model: "text-embedding-3-small" })
const vectorstoreAnkush = await Chroma.fromTexts(textsAnkush, {}, embeddingsAnkush, {
  collectionName: "ankush"
})
const retrieverAnkush = vectorstoreAnkush.asRetriever(1)

## 查询分析

我们将使用函数调用来结构化输出。我们会让其返回多个查询。

In [5]:
import { z } from "zod";

const searchSchema = z.object({
    query: z.string().describe("Query to look up"),
    person: z.string().describe("Person to look things up for. Should be `HARRISON` or `ANKUSH`.")
})

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [ ]:
// @lc-docs-hide-cell
import { ChatOpenAI } from '@langchain/openai';

const llm = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
})

In [6]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableSequence, RunnablePassthrough } from "@langchain/core/runnables";

const system = `You have the ability to issue search queries to get information to help answer user information.`
const prompt = ChatPromptTemplate.fromMessages(
[
    ["system", system],
    ["human", "{question}"],
]
)
const llmWithTools = llm.withStructuredOutput(searchSchema, {
name: "Search"
})
const queryAnalyzer = RunnableSequence.from([
    {
        question: new RunnablePassthrough(),
    },
    prompt,
    llmWithTools
])

我们可以看到，这允许在检索器之间进行路由

In [7]:
await queryAnalyzer.invoke("where did Harrison Work")

{ query: "workplace of Harrison", person: "HARRISON" }

In [8]:
await queryAnalyzer.invoke("where did ankush Work")

{ query: "Workplace of Ankush", person: "ANKUSH" }

## 使用查询分析进行检索

那么我们如何在链中包含这一点呢？我们只需要一些简单的逻辑来选择检索器并传入搜索查询

In [9]:
const retrievers = {
    HARRISON: retrieverHarrison,
    ANKUSH: retrieverAnkush,
}

In [17]:
import { RunnableConfig, RunnableLambda } from "@langchain/core/runnables";

const chain = async (question: string, config?: RunnableConfig) => {
    const response = await queryAnalyzer.invoke(question, config);
    const retriever = retrievers[response.person];
    return retriever.invoke(response.query, config);
}

const customChain = new RunnableLambda({ func: chain });

In [18]:
await customChain.invoke("where did Harrison Work")

[ Document { pageContent: "Harrison worked at Kensho", metadata: {} } ]

In [19]:
await customChain.invoke("where did ankush Work")

[ Document { pageContent: "Ankush worked at Facebook", metadata: {} } ]

## 下一步
你现在已经了解了一些在查询分析系统中处理多个检索器的技术。

接下来，查看本节中其他一些查询分析指南，例如[如何处理未生成查询的情况](/docs/how_to/query_no_queries)。